    SCRIPT:  trainline-senior_data_scientist-case_study-aysha_streeter

    AUTHOR:  Aysha Streeter <aysha.streeter@outlook.com>

    DATE:    25th November 2025 @ 15:30 to 16:30

# LIBRARIES

In [1]:
# General
import pandas as pd
import numpy as np
import json

# Visualisation
from IPython.display import IFrame

# Modelling
from sklearn.neighbors import NearestNeighbors
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

# AIM

In [2]:
# Interview task pdf
IFrame('Data Scientist - Case Study Task.pdf', width = 900, height = 300)

# SETTINGS

In [3]:
# Settings
# Displaying all columns.
pd.set_option('display.max_columns', None)
pd.options.display.max_colwidth = 1000
# Displaying all rows.
pd.set_option('display.max_rows', 1000)

# IMPORT

In [4]:
# Importing provided datasets.
df_sales = pd.read_csv('sales.csv', index_col = 0)
df_stations = pd.read_csv('stations.csv', index_col = 0)

# Importing lookups.
df_postcodes = pd.read_csv('lookups/lookup_postcodes.csv', low_memory = False)
df_coastal = pd.read_csv('lookups/lookup_coastal.csv')
df_bua =  pd.read_csv('lookups/lookup_bua.csv', encoding = 'cp1252', low_memory = False)
df_rgn = pd.read_csv('lookups/lookup_rgn.csv')
df_ru11ind = pd.read_csv('lookups/lookup_ru11ind.csv')
with open("lookups/lookup_holidays.json") as f:
    data_holidays = json.load(f)
with open("lookups/lookup_strikes.json") as f:
    data_strikes = json.load(f)

# QUALITY CHECKS

In [5]:
# Writing a list of dataframes so relevant checks applied to both.
dataframes = [df_sales, df_stations]

In [6]:
# Quick peek at what's in the dataframes.
for df in dataframes:
    print(df.head(), '\n')

         date        sales                station
0  2023-01-01  1346.840438  Manchester Piccadilly
1  2023-01-02  1347.696314  Manchester Piccadilly
2  2023-01-03  1349.757360  Manchester Piccadilly
3  2023-01-04  1354.646786  Manchester Piccadilly
4  2023-01-05  1348.356770  Manchester Piccadilly 

       station    lat   lon       operator
0   Abbey Wood  51.49  0.12   English Rail
1     Aberdeen  57.14 -2.09  Scottish Rail
2  Abergavenny  51.81 -3.00     Welsh Rail
3  Aberystwyth  52.41 -4.08     Welsh Rail
4   Accrington  53.75 -2.36   English Rail 



In [7]:
# Getting basic info re: whether null values, data types abd volume.
# WARNING!  There is a date column with data type "object"; this will be converted.
for df in dataframes:
    print(f"{df.info()}\n")

<class 'pandas.core.frame.DataFrame'>
Index: 70100 entries, 0 to 70099
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   date     70100 non-null  object 
 1   sales    70100 non-null  float64
 2   station  70100 non-null  object 
dtypes: float64(1), object(2)
memory usage: 2.1+ MB
None

<class 'pandas.core.frame.DataFrame'>
Index: 1260 entries, 0 to 1259
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   station   1260 non-null   object 
 1   lat       1260 non-null   float64
 2   lon       1260 non-null   float64
 3   operator  1260 non-null   object 
dtypes: float64(2), object(2)
memory usage: 49.2+ KB
None



In [8]:
# Converting date from object to date.
df_sales['date'] = pd.to_datetime(df_sales['date'])

In [9]:
# Above indicates no None/Nan values, but there may still be blanks or other placeholders indicating missing data.
# There doesn't seem to be any unusual values in the df_sales station 
for df in dataframes:
    for col in df.columns:
        if df[col].dtype == 'object':
            print(f'column_name: {df[col].name}\n sorted_values: {(df[col].sort_values().unique())}\n unique_values: {df[col].nunique()}\n')

column_name: station
 sorted_values: ['Aberdeen' 'Ashford International' 'Banbury' 'Basingstoke' 'Bath Spa'
 'Bedford' 'Birmingham International' 'Birmingham New Street'
 'Bishops Stortford' 'Bournemouth' 'Brighton' 'Bristol Parkway'
 'Bristol Temple Meads' 'Cambridge' 'Cardiff Central' 'Carlisle'
 'Chelmsford' 'Cheltenham Spa' 'Chester' 'Chesterfield' 'Chippenham'
 'Clapham Junction' 'Colchester' 'Coventry' 'Crewe' 'Darlington'
 'Didcot Parkway' 'Doncaster' 'Durham' 'East Croydon'
 'Edinburgh (Waverley)' "Exeter St David's" 'Gatwick Airport'
 'Glasgow Central' 'Glasgow Queen Street' 'Gloucester' 'Grantham'
 'Guildford' 'Haymarket' 'Haywards Heath' 'Hull' 'Ipswich' 'Kettering'
 'Lancaster' 'Leamington Spa' 'Leeds' 'Leicester' 'Lincoln'
 'Liverpool Lime Street' 'London Bridge' 'London Euston'
 'London Kings Cross' 'London Liverpool Street' 'London Marylebone'
 'London Paddington' 'London St Pancras International' 'London Victoria'
 'London Waterloo' 'Loughborough' 'Macclesfield' 'Manche

In [10]:
# Getting unique dates from df and for indiv stations printing if any dates are missing.
all_dates = df_sales['date'].unique()
missing_dates = (df_sales.groupby('station')['date'].apply(lambda x: set(all_dates) - set(x)))
missing_dates

station
Aberdeen                           {}
Ashford International              {}
Banbury                            {}
Basingstoke                        {}
Bath Spa                           {}
Bedford                            {}
Birmingham International           {}
Birmingham New Street              {}
Bishops Stortford                  {}
Bournemouth                        {}
Brighton                           {}
Bristol Parkway                    {}
Bristol Temple Meads               {}
Cambridge                          {}
Cardiff Central                    {}
Carlisle                           {}
Chelmsford                         {}
Cheltenham Spa                     {}
Chester                            {}
Chesterfield                       {}
Chippenham                         {}
Clapham Junction                   {}
Colchester                         {}
Coventry                           {}
Crewe                              {}
Darlington                         {}
Didc

In [11]:
# Basic stats and looking for flags.
for df in dataframes:
    print(f"{df.describe(include = 'all')}\n")

                                 date         sales                station
count                           70100  70100.000000                  70100
unique                            NaN           NaN                    100
top                               NaN           NaN  Manchester Piccadilly
freq                              NaN           NaN                    701
mean    2023-12-17 00:00:00.000000256    337.535624                    NaN
min               2023-01-01 00:00:00     92.314156                    NaN
25%               2023-06-25 00:00:00    135.779942                    NaN
50%               2023-12-17 00:00:00    200.444485                    NaN
75%               2024-06-09 00:00:00    371.568482                    NaN
max               2024-12-01 00:00:00   2754.247272                    NaN
std                               NaN    390.673461                    NaN

               station          lat          lon      operator
count             1260  1260.000000 

Observations:
+ Date range is for two years from 01/01/2023 to 1/12/2024.  n.b. month of December 2024 not present.
+ 2024 has a leap year.  n.b. for prediction.
+ Large range of sales, with standard deviation larger than the mean.
+ Mean larger than median, indicating right skew distribution and possibility of outliers.
+ Looks like each of the 100 stations appears 701 times, but needs verifying.
+ The lookups have a duplicate, in exeter central, but this is not relevant to the df_sales stations list.
+ Latitudes and longitudes seem sensible for GB.

In [12]:
# Checking for duplicates.
# Whether same row of data is present more than once.
for df in dataframes:
    print('Identical entries: \n', df[df.duplicated()], '\n')

Identical entries: 
 Empty DataFrame
Columns: [date, sales, station]
Index: [] 

Identical entries: 
 Empty DataFrame
Columns: [station, lat, lon, operator]
Index: [] 



In [13]:
# Tells us, not identical duplicated entry for exeter central, but two locations called this, with differnt info.
# Google search indicates English Rail value is correct, so other removed from look ups.
print(df_stations[df_stations['station']=='Exeter Central'], '\n')
df_stations = df_stations[~((df_stations['station']=='Exeter Central') & (df_stations['operator']=='Trainline'))]
# Checking removal.
print(df_stations[df_stations['station']=='Exeter Central'])

            station    lat   lon      operator
388  Exeter Central  51.46 -0.90     Trainline
389  Exeter Central  50.72 -3.53  English Rail 

            station    lat   lon      operator
389  Exeter Central  50.72 -3.53  English Rail


# FEATURE ENGINEERING

In [14]:
# Some formatting amendments to prevent future issues.
# Mainly removal of punctuation, case folding and removing spaces.
df_sales['station_adj'] = df_sales['station'].str.lower().str.strip().str.replace('(', '').str.replace("'", '').str.replace(' ', '_').str.replace(')', '').str.replace('-', '_')

## GEODATA

### DERIVING CODES FROM LONGITUDE-LATITUDE

In [15]:
# Obtaining region, rurality and built up area code from lookups, based on nearest neighbours algorithm across longitudes and latitudes.
# Would enable visualisation and analysis for region (or another geo level of choosing from postcodes lookup).
# Would also allow matching with other third party data.

# Renaming column, so aligned with dataset.
df_postcodes = df_postcodes.rename(columns={"long": "lon"})

# Using nearest neighbours.
# Only want closest single value from lookup, so n_neighbours = 1
# ball_tree best for continuous numeric data, as lon lat is; along with high dimensionality, its faster.
nbrs = NearestNeighbors(n_neighbors = 1, algorithm = 'ball_tree').fit(df_postcodes[['lat','lon']])
distances, indices = nbrs.kneighbors(df_stations[['lat','lon']])

#  Adding the requested columns to the df_stations lookup.
df_stations['rgn'] = df_postcodes.iloc[indices.flatten()]['rgn'].values
df_stations['ru11ind'] = df_postcodes.iloc[indices.flatten()]['ru11ind'].values


C:\Users\aysha\AppData\Local\Temp\ipykernel_25828\3200932652.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stations['rgn'] = df_postcodes.iloc[indices.flatten()]['rgn'].values
C:\Users\aysha\AppData\Local\Temp\ipykernel_25828\3200932652.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stations['ru11ind'] = df_postcodes.iloc[indices.flatten()]['ru11ind'].values


In [16]:
# Performing similar task, from a built up area lookup to determine whether area is coastal.

# Renaming column, so aligned with dataset.
df_bua.reset_index(inplace = True)
df_bua.rename(columns={"LONG": "lon", "LAT": "lat"}, inplace= True)

# Only want closest single value from lookup, so n_neighbours = 1
# ball_tree best for continuous numeric data, as lon lat is; along with high dimensionality, its faster.
nbrs = NearestNeighbors(n_neighbors = 1, algorithm = 'ball_tree').fit(df_bua[['lat','lon']])
distances, indices = nbrs.kneighbors(df_stations[['lat','lon']])

#  Adding the requested columns to the df_stations lookup.
df_stations['bua'] = df_bua.iloc[indices.flatten()]['BUA22CD'].values

C:\Users\aysha\AppData\Local\Temp\ipykernel_25828\1783026704.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stations['bua'] = df_bua.iloc[indices.flatten()]['BUA22CD'].values


In [17]:
# Current state of df after matching.
df_stations.head()

,station,lat,lon,operator,rgn,ru11ind,bua
0,Abbey Wood,51.49,0.12,English Rail,E12000007,A1,E63004992
1,Aberdeen,57.14,-2.09,Scottish Rail,S99999999,1,S45000002
2,Abergavenny,51.81,-3.00,Welsh Rail,W99999999,F1,W45000324
3,Aberystwyth,52.41,-4.08,Welsh Rail,W99999999,C2,W45000228
4,Accrington,53.75,-2.36,English Rail,E12000002,C1,E63000864


In [18]:
# Quick copy, to save me doing above again, in case I balls below up.
df_x = df_stations.copy()

### MAPPING CODES TO NAMES

In [19]:
# REGION
# Bringing through region name to station table.
# Tidying by renaming, dropping and formatting columns.
df_stations = df_stations.merge(df_rgn.reset_index()[['RGN20CD', 'RGN20NM']], left_on = 'rgn', right_on = 'RGN20CD', how = 'left')
df_stations.rename(columns = {'RGN20NM': 'region_nm'}, inplace = True)
df_stations.drop(['RGN20CD'], axis = 1, inplace = True)
df_stations['region_nm'] = df_stations['region_nm'].str.lower().str.replace('(pseudo) ', '').str.replace(' ', '_')

# RURAL-URBAN
# Bringing through name of rural-urban classification.
# Remappping urban and rural to unify GB and for fewer groupings.
df_stations = df_stations.merge(df_ru11ind, left_on = 'ru11ind', right_on = 'RU11IND', how = 'left')
df_stations.rename(columns = {'RU11NM': 'rurality_nm'}, inplace = True)
df_stations = df_stations.drop(columns=['RU11IND'])
df_stations['rurality_nm'] = np.where(df_stations['rurality_nm'].isin(['(England/Wales) Urban major conurbation', '(Scotland) Large Urban Area']), 'urban_major_conurbation',
                                      np.where(df_stations['rurality_nm'] == '(England/Wales) Urban minor conurbation', 'urban_minor_conurbation',
                                            np.where(df_stations['rurality_nm'] == '(England/Wales) Urban city and town', 'urban_city_town',
                                                    np.where(df_stations['rurality_nm'].isin(['(England/Wales) Rural hamlet and isolated dwellings', '(England/Wales) Rural village']), 'rural', 'unknown'))))
                                                                                              
# Obtaining whether area is coastal or not using bua code.
# Data only available for England and Wales, so -1 for Scotland.
df_stations['coastal_flag'] = np.where(df_stations['bua'].isin(df_coastal['BUA code']), 1,
                                       np.where(df_stations['region_nm'] == 'scotland', -1, 0))

# # Bringing through operator, region name, rurality code and coastal flag from station table to df_sales.
df_sales = df_sales.merge(df_stations[['station', 'operator','region_nm', 'lat', 'lon', 'rurality_nm','coastal_flag']], on = 'station', how = 'left')
df_sales['operator'] = df_sales['operator'].str.replace(' ', '_').str.lower()

In [20]:
# Peek at sales table.
df_sales.head()

,date,sales,station,station_adj,operator,region_nm,lat,lon,rurality_nm,coastal_flag
0,2023-01-01,1346.840438,Manchester Piccadilly,manchester_piccadilly,english_rail,north_west,53.47,-2.23,urban_major_conurbation,0
1,2023-01-02,1347.696314,Manchester Piccadilly,manchester_piccadilly,english_rail,north_west,53.47,-2.23,urban_major_conurbation,0
2,2023-01-03,1349.757360,Manchester Piccadilly,manchester_piccadilly,english_rail,north_west,53.47,-2.23,urban_major_conurbation,0
3,2023-01-04,1354.646786,Manchester Piccadilly,manchester_piccadilly,english_rail,north_west,53.47,-2.23,urban_major_conurbation,0
4,2023-01-05,1348.356770,Manchester Piccadilly,manchester_piccadilly,english_rail,north_west,53.47,-2.23,urban_major_conurbation,0


## DATES

### GRANULARITY

In [21]:
# Creating columns from date to indicate year, month, day of week, whether weekend and number of the week.
# Intending to assist analysis and visualisation.
df_sales['year'] = df_sales['date'].dt.year
df_sales['month'] = df_sales['date'].dt.month
df_sales['week_number'] = df_sales['date'].dt.isocalendar().week.astype('int')
df_sales['day'] = df_sales['date'].dt.day
df_sales['month_day'] = df_sales['date'].dt.strftime('%m-%d')  
df_sales['week_day'] = (df_sales['date'].dt.day_name()).str.lower()
df_sales['weekend_flag'] = df_sales['week_day'].isin(['saturday', 'sunday']).astype(int)

In [22]:
# Peek at sales table.
df_sales.head()

,date,sales,station,station_adj,operator,region_nm,lat,lon,rurality_nm,coastal_flag,year,month,week_number,day,month_day,week_day,weekend_flag
0,2023-01-01,1346.840438,Manchester Piccadilly,manchester_piccadilly,english_rail,north_west,53.47,-2.23,urban_major_conurbation,0,2023,1,52,1,01-01,sunday,1
1,2023-01-02,1347.696314,Manchester Piccadilly,manchester_piccadilly,english_rail,north_west,53.47,-2.23,urban_major_conurbation,0,2023,1,1,2,01-02,monday,0
2,2023-01-03,1349.757360,Manchester Piccadilly,manchester_piccadilly,english_rail,north_west,53.47,-2.23,urban_major_conurbation,0,2023,1,1,3,01-03,tuesday,0
3,2023-01-04,1354.646786,Manchester Piccadilly,manchester_piccadilly,english_rail,north_west,53.47,-2.23,urban_major_conurbation,0,2023,1,1,4,01-04,wednesday,0
4,2023-01-05,1348.356770,Manchester Piccadilly,manchester_piccadilly,english_rail,north_west,53.47,-2.23,urban_major_conurbation,0,2023,1,1,5,01-05,thursday,0


### EVENTS

In [23]:
# Bank holidays
# Creating dataframe to flag bank holidays according to region.
# Those that have england-wales in region apply to all, whereas those with scotland in region are scotland only.

# Sorting json file to convert to dateframe, with columns for date, region and name of holiday.
bh_info = []
for region, years in data_holidays.items():
    for year, holidays in years.items():
        for holiday in holidays:
            bh_info.append({'title': holiday['title'],
                            'date': pd.to_datetime(holiday['date']),
                            'region': region})
            
df_holidays = pd.DataFrame(bh_info)

# Creating separate dfs for ew and scottish holidays.
holidays_scot = df_holidays[df_holidays['region'] == 'scotland']
holidays_ew = df_holidays[df_holidays['region'] == 'england-and-wales']

# If region is scotland, should flag for scottish plus englisha nd welsh holidays.
sales_scot = df_sales[df_sales['region_nm'] == 'scotland']
sales_scot = sales_scot.merge(pd.concat([holidays_scot, holidays_ew]), on = 'date', how = 'left')

# Removing duplicates.
sales_scot = sales_scot.drop_duplicates(subset=['station', 'date'])

# If region isn't scotland, should flag for english and welsh holidays only.
sales_ew = df_sales[df_sales['region_nm'] != 'scotland']
sales_ew = sales_ew.merge(holidays_ew, on = 'date', how = 'left')

# Recombining
df_sales = pd.concat([sales_ew, sales_scot], ignore_index=True)
df_sales['bank_holiday_flag'] = df_sales['title'].notna().astype(int)
df_sales.drop(['title', 'region'], axis = 1, inplace = True)

In [24]:
#  Similar to above for strikes.
#  To be revisited, ran out of time.
# # Strikes
# # Creating column that flags if the date was the date of an official train strike.
# # Limitation:  Does not account for region specific strikes.
# rows = []
# for entry in data_strikes["train_strikes"]:
#     for d in entry["dates"]:
#         rows.append({
#             "date": pd.to_datetime(d["date"]),  # use the string
#             "union": d["union"]
#         })

# df_strikes = pd.DataFrame(rows)
# df_sales['strike_flag'] = df_sales['date'].isin(df_strikes['date']).astype(int)

In [25]:
# Creating column to flag if working day, based on weekend and bank holiday flag.
df_sales['working_day'] = np.where(((df_sales['weekend_flag'] == 1)|(df_sales['bank_holiday_flag'] == 1)), 0, 1)

In [26]:
# state of df_sales.
df_sales.head()

,date,sales,station,station_adj,operator,region_nm,lat,lon,rurality_nm,coastal_flag,year,month,week_number,day,month_day,week_day,weekend_flag,bank_holiday_flag,working_day
0,2023-01-01,1346.840438,Manchester Piccadilly,manchester_piccadilly,english_rail,north_west,53.47,-2.23,urban_major_conurbation,0,2023,1,52,1,01-01,sunday,1,0,0
1,2023-01-02,1347.696314,Manchester Piccadilly,manchester_piccadilly,english_rail,north_west,53.47,-2.23,urban_major_conurbation,0,2023,1,1,2,01-02,monday,0,1,0
2,2023-01-03,1349.757360,Manchester Piccadilly,manchester_piccadilly,english_rail,north_west,53.47,-2.23,urban_major_conurbation,0,2023,1,1,3,01-03,tuesday,0,0,1
3,2023-01-04,1354.646786,Manchester Piccadilly,manchester_piccadilly,english_rail,north_west,53.47,-2.23,urban_major_conurbation,0,2023,1,1,4,01-04,wednesday,0,0,1
4,2023-01-05,1348.356770,Manchester Piccadilly,manchester_piccadilly,english_rail,north_west,53.47,-2.23,urban_major_conurbation,0,2023,1,1,5,01-05,thursday,0,0,1


In [ ]:
# Exporting raw df.
df_sales.to_csv('sales_processed.csv', index = 0)

# UPLIFT MODELLING - PROPENSITY SCORING

This is the preferred option for the following reasons:
+ Limited number of features to stop overfitting.
+ Gives an average treatment effect.
+ Not huge volume of data; only one point per day for the treatment.
+ Nottingham billboard choice with costs likely wasn't random.

However, a two-model approach might be better to target campaigns.

In [71]:
# Defining Nottingham as the treatment.
df_sales['treatment'] = (df_sales['station'] == 'Nottingham').astype(int)
# Getting pre-period dataframe section.
pre_period = df_sales[(df_sales['date'] >= '2023-10-01') & (df_sales['date'] < '2024-10-01')]

In [72]:
# Defining control groups.
df_uplift_east_midlands = pre_period[pre_period['region_nm'] == 'east_midlands']
df_uplift_mid_north = pre_period[(pre_period['region_nm'].isin(['east_midlands', 'west_midlands', 'yorkshire_and_the_humber', 'east_of_england']))]
df_uplift_mid_north_no_big = pre_period[(pre_period['region_nm'].isin(['east_midlands', 'west_midlands', 'yorkshire_and_the_humber', 'east_of_england'])) & (~pre_period['station_adj'].isin(['birmingham_new_street', 'leeds']))]

In [73]:
# Creating a model run that will run through each of the dataframes, preprocess the data, apply the xgboost model and generate uplift score.

# Creating list of dataframes
uplift_dataframes = [df_uplift_east_midlands, df_uplift_mid_north, df_uplift_mid_north_no_big]

# Defining features to be used.
variables = ['region_nm', 'coastal_flag', 'week_day', 'bank_holiday_flag']

# Loop running through each of the dataframes.
outputs = []
for df in uplift_dataframes:
    # One hot encodign categorical features.
    X = pd.get_dummies(df[variables], 
                       columns=['region_nm', 'week_day'], 
                       drop_first=True)
    y = df['treatment']

    # Deifning xgboost.
    xgb = XGBClassifier(eval_metric = 'logloss', random_state = 2025)

    # Adding a very small grid of hyperparameters.
    param_grid = {
        'n_estimators': [100, 200, 1000],
        'max_depth': [3, 4, 5],
        'learning_rate': [0.05, 0.1],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0]
    }

    # Adding a 5-fold cross-validation full grid search.
    grid_search = GridSearchCV(
        estimator=xgb,
        param_grid=param_grid,
        scoring='roc_auc',
        cv=5,
        verbose=0,
        n_jobs=-1)

    # Fitting the grid.
    grid_search.fit(X, y)

    # Predicting propensity scores
    best_xgb = grid_search.best_estimator_
    df['propensity_score'] = best_xgb.predict_proba(X)[:, 1]

    train_auc = roc_auc_score(y, best_xgb.predict_proba(X)[:,1])
    print("training ROC-AUC:", train_auc)
    print("cv ROC-AUC:", grid_search.best_score_)

    # Splitting into treated and control
    treated = df[df['treatment'] == 1]
    control = df[df['treatment'] == 0]

    # Nearest neighbor matching
    nn = NearestNeighbors(n_neighbors=1)
    nn.fit(control[['propensity_score']])
    distances, indices = nn.kneighbors(treated[['propensity_score']])
    matched_controls = control.iloc[indices.flatten()]

    # Campaign period sales
    campaign_period = df_sales[(df_sales['date'] >= '2024-10-01') & (df_sales['date'] <= '2024-12-01')]
    treated_sales = campaign_period[campaign_period['station'] == 'Nottingham']['sales'].mean()
    control_sales = campaign_period[campaign_period['station'].isin(matched_controls['station'])]['sales'].mean()

    # Calculating uplift
    uplift = treated_sales - control_sales
    uplift_perc = (uplift / control_sales) * 100 if control_sales != 0 else None
        
    outputs.append({
        'treatment sales': treated_sales,
        'control sales': control_sales,
        'uplift': uplift,
        'uplift_perc' : uplift_perc,
        'cv_roc': grid_search.best_score_
    })

C:\Users\aysha\AppData\Local\Temp\ipykernel_24764\3541405557.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['propensity_score'] = best_xgb.predict_proba(X)[:, 1]


training ROC-AUC: 0.49999999999999994
cv ROC-AUC: 0.4820319626278883


C:\Users\aysha\AppData\Local\Temp\ipykernel_24764\3541405557.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['propensity_score'] = best_xgb.predict_proba(X)[:, 1]


training ROC-AUC: 0.8714285714285714
cv ROC-AUC: 0.8711163153786103


C:\Users\aysha\AppData\Local\Temp\ipykernel_24764\3541405557.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['propensity_score'] = best_xgb.predict_proba(X)[:, 1]


training ROC-AUC: 0.8636363636363636
cv ROC-AUC: 0.862155136801903


In [75]:
# Convert to summary DataFrame
summary_df = pd.DataFrame(outputs)
print(summary_df)

   treatment sales  control sales      uplift  uplift_perc    cv_roc
0       566.487517     236.078161  330.409356   139.957611  0.482032
1       566.487517     401.382577  165.104940    41.134058  0.871116
2       566.487517     401.382577  165.104940    41.134058  0.862155


# BASIC ANALYSIS

## PROPORTIONS OF SALES BY GROUPINGS

In [29]:
grouping = ['operator', 'region_nm', 'station']
dataframe = {}
for group in grouping:
    df_group = pd.DataFrame(df_sales.groupby(group)['sales'].sum().div(df_sales['sales'].sum()).mul(100)).reset_index().sort_values(by='sales', ascending=False).reset_index(drop=True)
    dataframe[group] = df_group
    # print(dataframe[group], '\n')

df_operator = dataframe['operator']
df_rgn_nm  = dataframe['region_nm']
df_station = dataframe['station']

In [ ]:
df_operator

,operator,sales
0,english_rail,92.550537
1,scottish_rail,5.425400
2,welsh_rail,2.024063


## PROPORTIONS OF SALES BY YEAR

In [32]:
# Only one day in December 2024, so removing December from comparisions.
df_sales_no_dec = df_sales[df_sales['month'] != 12]

In [33]:
# Getting annual sales by year with % change.
# Grouping by month and year to get total sales.
annual_sales = df_sales_no_dec.groupby(['year'])['sales'].sum().reset_index()

# Pivoting for annual difference
pivot_sales = annual_sales.set_index('year').T

# Calculating % change.
pivot_sales['change_perc'] = 100 * (pivot_sales[2024] / pivot_sales[2023] - 1)
pivot_sales.reset_index(inplace = True)
pivot_sales

year,index,2023,2024,change_perc
0,sales,1.111541e+07,1.148198e+07,3.297878


In [34]:
# Getting annual sales by year with % change, broken down by groupings.
grouping = ['operator', 'region_nm', 'station']
dataframes = {}

for group in grouping:
    annual_sales = (df_sales_no_dec.groupby([group, 'year'])['sales'].mean().reset_index())
    pivot_sales = annual_sales.pivot_table(index=group,columns='year',values='sales')
    pivot_sales['change_perc'] = ((pivot_sales[2024] / pivot_sales[2023] - 1) * 100)
    pivot_sales.reset_index(inplace=True)
    dataframes[group] = pivot_sales

df_operator = dataframes['operator']
df_rgn_nm   = dataframes['region_nm']
df_station  = dataframes['station']

In [35]:
df_rgn_nm

year,region_nm,2023,2024,change_perc
0,east_midlands,195.806508,203.991654,4.180222
1,east_of_england,229.678026,235.113540,2.366580
2,london,804.218862,818.808929,1.814191
3,north_east,382.420941,414.159194,8.299298
4,north_west,307.617981,316.547014,2.902637
5,scotland,358.382239,364.172628,1.615702
6,south_east,230.011298,237.797558,3.385164
7,south_west,225.777191,234.512263,3.868891
8,wales,223.433987,232.892853,4.233405
9,west_midlands,279.949848,291.511041,4.129737


## PROPORTIONS OF SALES BY MONTH

In [36]:
# Getting monthly sales by year with % change.
# Grouping by month and year to get total sales.
monthly_sales = df_sales_no_dec.groupby(['year','month'])['sales'].mean().reset_index()

# Pivoting for annual difference
pivot_sales = monthly_sales.pivot(index = 'month', columns = 'year', values = 'sales')

# Calculating % change.
pivot_sales['change_perc'] = 100 * (pivot_sales[2024] / pivot_sales[2023] - 1)
pivot_sales.reset_index(inplace = True)
pivot_sales

year,month,2023,2024,change_perc
0,1,320.766739,330.478114,3.027550
1,2,322.137457,330.635116,2.637899
2,3,326.802896,334.168037,2.253695
3,4,332.044709,339.790915,2.332880
4,5,335.940299,344.782236,2.631996
5,6,337.866271,347.072367,2.724775
6,7,338.131506,347.417067,2.746139
7,8,337.555296,347.919333,3.070323
8,9,336.896102,349.653699,3.786804
9,10,336.856948,350.066990,3.921558
